In [ ]:
!pip install efficientnet_pytorch

In [ ]:
!pip install albumentations==0.4.6

In [ ]:
import os
import sys
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tqdm import tqdm
import torch
import torchvision.models as models
import torch.nn as nn
from efficientnet_pytorch import model as enet
import random
from sklearn.model_selection import KFold, StratifiedKFold
import gc
from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage
import itertools
# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
import typing as tp
import itertools
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from tensorflow.python.framework import ops
 
gc.collect()
torch.cuda.empty_cache()
 
def set_seed(seed = 0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state
 
seed = 42
random_state = set_seed(seed)
 
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
def get_valid_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(1024, 512),
        ]
    )
 
class ClassificationDataset:
    
    def __init__(self, image_paths, targets, trans = None): 
        self.image_paths = image_paths
        self.targets = targets
        self.transform = trans
 
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item])[[0, 2, 4]]
        image = image.astype(np.float32)
        image = np.vstack(image).transpose((1, 0)).astype(np.float32)
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        targets = self.targets[item] 
        image=image[np.newaxis, ]    
        return {
            "image": torch.tensor(image.copy(), dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_pretrained(backbone)   
        
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=3, bias=False)
 
    def extract(self, x):
        return self.enet(x)
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        
        return x
        
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs1 = data["image"]
            targets = data["targets"]
            
            inputs1 = inputs1.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs1)
            output = torch.sigmoid(output)
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

submission=pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path']=submission['id'].apply(lambda x:f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

baseline_path0 = "../input/etcv0/"
baseline_path1 = "../input/etcv1/"
baseline_name0 = 'efficientnet-b0'
baseline_name1 = 'efficientnet-b1'
models = []

#Load effnet models
model1_0 = enetv2(baseline_name0, out_dim=1)
model1_0.load_state_dict(torch.load(baseline_path0+"efficientnet-b0-0.pt"))
model1_0.to(device)
models.append(model1_0)
model1_1 = enetv2(baseline_name0, out_dim=1)
model1_1.load_state_dict(torch.load(baseline_path0+"efficientnet-b0-1.pt"))
model1_1.to(device)
models.append(model1_1)
model1_2 = enetv2(baseline_name0, out_dim=1)
model1_2.load_state_dict(torch.load(baseline_path0+"efficientnet-b0-2.pt"))
model1_2.to(device)
models.append(model1_2)
model1_3 = enetv2(baseline_name0, out_dim=1)
model1_3.load_state_dict(torch.load(baseline_path0+"efficientnet-b0-3.pt"))
model1_3.to(device)
models.append(model1_3)
model1_4 = enetv2(baseline_name0, out_dim=1)
model1_4.load_state_dict(torch.load(baseline_path0+"efficientnet-b0-4.pt"))
model1_4.to(device)
models.append(model1_4)
test_dataset=ClassificationDataset(image_paths=submission.img_path.values, targets=submission.target.values, trans = get_valid_transforms())
test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=12,shuffle=False,num_workers=4)

sig=torch.nn.Sigmoid()
outs=[]
weights=[0.25,0.25,0.25,0.25,0.25]
i=0
for model in models:
    predictions,valid_targets=evaluate(test_loader, model, device=device)
    predictions=np.array(predictions)[:,0]
    predictions=predictions*weights[i]
    i+=1
    outs.append(predictions)
    
#Load effnet models
models = []
model1_5 = enetv2(baseline_name1, out_dim=1)
model1_5.load_state_dict(torch.load(baseline_path1+"efficientnet-b1-0.pt"))
model1_5.to(device)
models.append(model1_5)
model1_6 = enetv2(baseline_name1, out_dim=1)
model1_6.load_state_dict(torch.load(baseline_path1+"efficientnet-b1-1.pt"))
model1_6.to(device)
models.append(model1_6)
model1_7 = enetv2(baseline_name1, out_dim=1)
model1_7.load_state_dict(torch.load(baseline_path1+"efficientnet-b1-2.pt"))
model1_7.to(device)
models.append(model1_7)
model1_8 = enetv2(baseline_name1, out_dim=1)
model1_8.load_state_dict(torch.load(baseline_path1+"efficientnet-b1-3.pt"))
model1_8.to(device)
models.append(model1_8)
model1_9 = enetv2(baseline_name1, out_dim=1)
model1_9.load_state_dict(torch.load(baseline_path1+"efficientnet-b1-4.pt"))
model1_9.to(device)
models.append(model1_9)

test_dataset=ClassificationDataset(image_paths=submission.img_path.values, targets=submission.target.values, trans = get_valid_transforms())
test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=10,shuffle=False,num_workers=4)

weights=[-0.05,-0.05,-0.05,-0.05,-0.05]
i=0
for model in models:
    predictions,valid_targets=evaluate(test_loader, model, device=device)
    predictions=np.array(predictions)[:,0]
    predictions=predictions*weights[i]
    i+=1
    outs.append(predictions)

#Simple mean for final value
pred=np.sum(np.array(outs),axis=0)
submission.target=pred
submission.drop(['img_path'],axis=1,inplace=True)
submission.to_csv('submission.csv', index=False)